# NEAR Analysis

NEAR is a proof-of-stake layer-1 blockchain that utilizes a unique sharding approach to address scalability and throughput challenges that plaque other chains.  Due to it being a non-EVM chain, data tends to be more sparse and challenging to collect, in contrast to EVM-compatible chains. Additionally, a separate smart-contract platform with EVM capabilitiy called Aurora was built on top of the NEAR blockchain, which provides a connection between NEAR and the Ethereum ecosystem. Aurora has played a significant role in the growth of the NEAR ecosystem, where it almost rivals its predecessor in size.

In this notebook I showcase different methods for extracting NEAR/Aurora data from various types of sources, and create visuals that are not widely used. This includes:

1. TVL comparison across top 5 protocols in NEAR and Aurora - (source: DefiLlama)
2. Bridging activity to NEAR in relation to other chains (source: Dune Dashboards)
3. Metapool staking trend (source: DappLooker)
4. Extra explorattion: Utilize subgraphs to get the number of daily new accounts

## 0. Install dependencies

In [3]:
from datetime import datetime
import pandas as pd
import time
from io import StringIO
import matplotlib.pyplot as plt
import requests
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from duneanalytics import DuneAnalytics
import asyncio


In [4]:
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# Metric 1: TVL comparison across NEAR & Aurora

Total value locked is a useful guage for determining which protocols are experiencing the highest level of engagement and volume. There is not much insight available on the web that compares the different protocols within Near/Aurora, so this section focuses on understanding which protocols actually make up the majority of TVL in the ecosystems, as well as analyzing what trends are currently taking shape.

Chart:
- Displays the TVL of the top 5 protocols on Aurora and NEAR. 
- In the month of May, Aurora's TVL was almost double that of NEARs, but that rise was shortlived as it quickly decreased to be even lower than NEARs (NEAR: 300m vs Aurora: 250m) 
- Ref Finance, the main DEX on NEAR, has seen fairly consistent TVL even with the recent bear market

In [2]:
defillama_base_url = "https://api.llama.fi"

In [3]:
# get tvl for chains: Near and Aurora
def get_defillama_chain_data(chain):
    """
    Get the TVL for a chain
    
    
    """
    url = f'{defillama_base_url}/charts/{chain}'
    try:
        request = requests.get(url).json()
    except request.ConnectionError as e:
        logging.error(f"There was an error with the request, {e}")
        sys.exit(1)
    
    df = pd.DataFrame(request)
    
    df['date'] = pd.to_datetime(df['date'], unit='s')
    
    df.set_index('date', inplace=True)
    df.index = df.index.date

    #add identifying prefix to col and concatenate to final df
    col_prefix = f"{chain}_"
    df = df.add_prefix(col_prefix)
    return df

near_df = get_defillama_chain_data('Near')
#near_df
aurora_df = get_defillama_chain_data('Aurora')
aurora_df

,Aurora_totalLiquidityUSD
2021-11-04,1.741316e+06
2021-11-05,1.843287e+06
2021-11-06,1.828659e+06
2021-11-07,2.079838e+06
2021-11-08,2.083266e+06
...,...
2022-07-03,2.548545e+08
2022-07-04,2.540264e+08
2022-07-05,2.577915e+08
2022-07-06,2.551577e+08


In [4]:
# get list of protocols in Aurora and Near chain
def get_defillama_protocol_data(protocol_slug: str):
    url = f'{defillama_base_url}/protocols'
    try:
        request = requests.get(url).json()
    except request.ConnectionError as e:
        logging.error(f"There was an error with the request, {e}")
        sys.exit(1)
    
    df = pd.DataFrame(request)
    
    df = df.loc[df['chain'] == protocol_slug]
    
#     df['date'] = pd.to_datetime(df['date'], unit='s')
    return df

near_protocols = get_defillama_protocol_data("Near")
# near_protocols

aurora_protocols = get_defillama_protocol_data("Aurora")
aurora_protocols


,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,mcap,parentProtocol,staking,pool2,language,openSource,forkedFrom,listedAt,audit,audits_link
83,1518,Bastion,-,BSTN,https://app.bastionprotocol.com/,The Liquidity Foundation of Aurora. Stableswap...,Aurora,https://icons.llama.fi/bastion.svg,0,None,...,3259228.0,NaN,NaN,NaN,NaN,NaN,[Compound],1.646686e+09,NaN,NaN
177,784,Trisolaris,aurora:0xFa94348467f64D5A457F75F8bc40495D33c65aBB,TRI,https://www.trisolaris.io/#/swap,"Trisolaris is #1 Dex on the Aurora engine, an ...",Aurora,https://icons.llama.fi/trisolaris.jpg,0,None,...,726183.0,NaN,NaN,NaN,NaN,NaN,[Uniswap],1.636045e+09,NaN,NaN
215,1492,Aurigami,aurora:0x09c9d464b58d96837f8d8b6f4d9fe4ad408d3a4f,PLY,https://www.aurigami.finance,"MultiPLY with Aurigami, the noncustodial lendi...",Aurora,https://icons.llama.fi/aurigami.png,2,None,...,1725426.0,NaN,NaN,NaN,NaN,NaN,[Compound],1.646169e+09,NaN,NaN
537,988,Vaporwave,aurora:0x2451dB68DeD81900C4F16ae1af597E9658689734,VWAVE,https://www.vaporwave.farm,Vaporwave Finance is a yield optimizer on the ...,Aurora,https://icons.llama.fi/vaporwave.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,[Beefy Finance],1.638882e+09,NaN,NaN
601,1136,Rose,aurora:0xdcD6D4e2B3e1D1E1E6Fa8C21C8A323DcbecfF970,ROSE,https://app.rose.fi,Rose is a liquidity protocol on Aurora compose...,Aurora,https://icons.llama.fi/rose.jpg,0,None,...,0.0,NaN,9.241355e+04,NaN,NaN,NaN,NaN,1.640738e+09,NaN,NaN
633,1227,Empyrean DAO,aurora:0xE9F226a228Eb58d408FdB94c3ED5A18AF6968fE1,EMPYR,https://empyrean.fi,The Decentralized Reserve Currency on Aurora,Aurora,https://icons.llama.fi/empyrean-dao.jpg,0,None,...,0.0,NaN,8.015017e+05,NaN,NaN,NaN,[Olympus DAO],1.641874e+09,NaN,NaN
647,1637,BlueBit,aurora:0x4148d2Ce7816F0AE378d98b40eB3A7211E1fcF0D,BBT,https://bluebit.fi/,BlueBit Finance is a yield aggregator built on...,Aurora,https://icons.llama.fi/bluebit.jpg,0,None,...,0.0,NaN,7.427197e+03,NaN,NaN,NaN,[],1.649686e+09,NaN,NaN
648,980,WannaSwap,0x7faA64Faf54750a2E3eE621166635fEAF406Ab22,WANNA,https://wannaswap.finance,WannaSwap is the Liquidity Central built on NE...,Aurora,https://icons.llama.fi/wannaswap.jpg,2,None,...,336601.0,NaN,9.007826e+04,NaN,NaN,NaN,[Uniswap],1.638703e+09,NaN,NaN
668,953,Nearpad,0xea7cc765ebc94c4805e3bff28d7e4ae48d06468a,PAD,https://www.nearpad.io,Launchpad & DeFi hub of the NEAR ecosystem.,Aurora,https://icons.llama.fi/nearpad.jpg,0,None,...,449417.0,NaN,NaN,NaN,NaN,NaN,[Uniswap],1.638393e+09,NaN,NaN
781,1705,PinkPea Finance,None,-,https://pinkpea.finance/,"PinkPea to enable everyone to safely access, c...",Aurora,https://icons.llama.fi/pinkpea-finance.png,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,[],1.651169e+09,NaN,NaN


In [5]:
near_protocols_slugs = ['ref-finance', 'burrow', 'meta-pool', 'linear-protocol', 'jumbo-exchange']
aurora_protocols_slugs = ['bastion', 'trisolaris', 'aurigami', 'dodo', 'synapse']


def get_all_protocol_tvls(chain_slug: str, protocol_slug_list: list):
    
    all_protocols_df = get_defillama_chain_data(chain_slug)
    
    for protocol in protocol_slug_list:
        url = f'{defillama_base_url}/protocol/{protocol}'
        try:
            request = requests.get(url).json()
        except request.ConnectionError as e:
            logging.error(f"There was an error with the request, {e}")
            sys.exit(1)
        request_tvl = request['chainTvls'][chain_slug]['tvl']

        protocol_df = pd.DataFrame(request_tvl)
        
        #convert date column and set as index
        protocol_df['date'] = pd.to_datetime(protocol_df['date'], unit='s')
        protocol_df.set_index('date', inplace=True)
        protocol_df.index = protocol_df.index.date
        
        #add identifying prefix to col and concatenate to final df
        col_prefix = f"{protocol}_"
        protocol_df = protocol_df.add_prefix(col_prefix)
        
        all_protocols_df = pd.concat([all_protocols_df, protocol_df], axis=1)
        
        #Calc % of near tvl
        tvl_col = f"{protocol}_totalLiquidityUSD"
        
        chain_tvl_col = f"{chain_slug}_totalLiquidityUSD"
        
        new_col = f"{protocol}_%_of_{chain_slug}_tvl"
        
        all_protocols_df[new_col] = all_protocols_df[tvl_col]/all_protocols_df[chain_tvl_col]
        
        
        
    all_protocols_df.sort_index(ascending=False, inplace=True)
        
    return all_protocols_df


near_protocols_tvl = get_all_protocol_tvls(chain_slug = "Near", protocol_slug_list = near_protocols_slugs)
#near_protocols_tvl

aurora_protocols_tvl = get_all_protocol_tvls(chain_slug = "Aurora", protocol_slug_list = aurora_protocols_slugs)

#aurora_protocols_tvl

In [6]:
# prepare the full dataset by combining the two dataframes
complete_ecosystem_data = pd.concat([aurora_protocols_tvl, near_protocols_tvl], axis=1)
complete_ecosystem_data

,Aurora_totalLiquidityUSD,bastion_totalLiquidityUSD,bastion_%_of_Aurora_tvl,trisolaris_totalLiquidityUSD,trisolaris_%_of_Aurora_tvl,aurigami_totalLiquidityUSD,aurigami_%_of_Aurora_tvl,dodo_totalLiquidityUSD,dodo_%_of_Aurora_tvl,synapse_totalLiquidityUSD,...,ref-finance_totalLiquidityUSD,ref-finance_%_of_Near_tvl,burrow_totalLiquidityUSD,burrow_%_of_Near_tvl,meta-pool_totalLiquidityUSD,meta-pool_%_of_Near_tvl,linear-protocol_totalLiquidityUSD,linear-protocol_%_of_Near_tvl,jumbo-exchange_totalLiquidityUSD,jumbo-exchange_%_of_Near_tvl
2022-07-07,2.561783e+08,1.513078e+08,0.590634,4.153830e+07,0.162146,2.760343e+07,0.107751,1.683235e+07,0.065706,4.540815e+06,...,2.042990e+08,0.666933,5.798337e+07,0.189286,3.080452e+07,0.100561,1.206406e+07,0.039383,1.088738e+06,0.003554
2022-07-06,2.551577e+08,1.508165e+08,0.591072,4.060588e+07,0.159140,2.715110e+07,0.106409,1.772230e+07,0.069456,4.651831e+06,...,2.010089e+08,0.666669,5.768057e+07,0.191304,3.006093e+07,0.099701,1.160497e+07,0.038489,1.072742e+06,0.003558
2022-07-05,2.577915e+08,1.515654e+08,0.587938,4.108377e+07,0.159368,2.745537e+07,0.106502,1.885910e+07,0.073156,4.599577e+06,...,1.989334e+08,0.660183,5.811168e+07,0.192850,3.110143e+07,0.103214,1.199680e+07,0.039813,1.100351e+06,0.003652
2022-07-04,2.540264e+08,1.496152e+08,0.588975,3.930081e+07,0.154711,2.815091e+07,0.110819,1.877243e+07,0.073900,4.516274e+06,...,1.947230e+08,0.665754,5.607394e+07,0.191716,2.924860e+07,0.100000,1.133013e+07,0.038737,1.026246e+06,0.003509
2022-07-03,2.548545e+08,1.508469e+08,0.591894,3.894920e+07,0.152829,2.812377e+07,0.110352,1.876914e+07,0.073647,4.556826e+06,...,1.896540e+08,0.657452,5.750953e+07,0.199362,2.895780e+07,0.100385,1.122268e+07,0.038904,1.040866e+06,0.003608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.116147e+06,1.347373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.283873e+05,7.116596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.164022e+05,50.308907,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.313749e+03,0.973701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#Plotting

fig1 = go.Figure()
fig1.add_trace(go.Scatter(
     x= complete_ecosystem_data.index, y = complete_ecosystem_data['ref-finance_totalLiquidityUSD'],
     name = 'Near - Ref Finance',
     mode = 'lines',
     line=dict(width=0.5, color='darkblue'),
     stackgroup = 'one'))
fig1.add_trace(go.Scatter(
     x= complete_ecosystem_data.index, y = complete_ecosystem_data['meta-pool_totalLiquidityUSD'],
     name = 'Near - Meta pool',
     mode = 'lines',
     line=dict(width=0.5,color='purple'),
     stackgroup = 'one'))
fig1.add_trace(go.Scatter(
     x= complete_ecosystem_data.index, y = complete_ecosystem_data['linear-protocol_totalLiquidityUSD'],
     name = 'Near - Linear Protocol',
     mode = 'lines',
     line=dict(width=0.5,color='darkgreen'),
     stackgroup = 'one'))
fig1.add_trace(go.Scatter(
     x= complete_ecosystem_data.index, y = complete_ecosystem_data['burrow_totalLiquidityUSD'],
     name = 'Near - Jumbo Exchange',
     mode = 'lines',
     line=dict(width=0.5,color='darkred'),
     stackgroup = 'one'))
fig1.add_trace(go.Scatter(
     x= complete_ecosystem_data.index, y = complete_ecosystem_data['jumbo-exchange_totalLiquidityUSD'],
     name = 'Near - Jumbo Exchange',
     mode = 'lines',
     line=dict(width=0.5,color='darkgray'),
     stackgroup = 'one'))

fig1.add_trace(go.Scatter(
     x= complete_ecosystem_data.index, y = complete_ecosystem_data['bastion_totalLiquidityUSD'],
     name = 'Aurora - Bastion',
     mode = 'lines',
     line=dict(width=0.5, color='lightyellow'),
     stackgroup = 'one'))
fig1.add_trace(go.Scatter(
     x= complete_ecosystem_data.index, y = complete_ecosystem_data['trisolaris_totalLiquidityUSD'],
     name = 'Aurora - Trisolaris',
     mode = 'lines',
     line=dict(width=0.5,color='lightpink'),
     stackgroup = 'one'))
fig1.add_trace(go.Scatter(
     x= complete_ecosystem_data.index, y = complete_ecosystem_data['aurigami_totalLiquidityUSD'],
     name = 'Aurora - Aurigami',
     mode = 'lines',
     line=dict(width=0.5,color='lightgreen'),
     stackgroup = 'one'))
fig1.add_trace(go.Scatter(
     x= complete_ecosystem_data.index, y = complete_ecosystem_data['dodo_totalLiquidityUSD'],
     name = 'Aurora - DODO',
     mode = 'lines',
     line=dict(width=0.5,color='pink'),
     stackgroup = 'one'))
fig1.add_trace(go.Scatter(
     x= complete_ecosystem_data.index, y = complete_ecosystem_data['synapse_totalLiquidityUSD'],
     name = 'Aurora - Synapse',
     mode = 'lines',
     line=dict(width=0.5,color='orange'),
     stackgroup = 'one'))

fig1.update_layout(
    title_text="TVL of Protocols in Aurora & NEAR"
)

fig1.show()

# Metric 2: Bridging into Near vs. competitor chains

Dune is a powerful tool for querying on-chain data, but it does not currently have a API which would allow for easier access outside of the Dune platform. Although, if data is available online there is most certainly exists some way to extract it, and in this case a 3rd party library is used to grab data from a Dune dashboard.

The data being explored is the total value locked by bridges from the Ethereum network. Cross-chain bridges operate by 'locking' assets on the chain where they originate from, and then new tokens are provided on the recieving blockchain. The bridges TVL is then a reflection of the amount of volume being bridged over to another chain.

Chart:
- This graph displays the TVL of chains that interact with Ethereum. Ethereum is by far the largest smart-contract platform, and a majority of bridging activity is performed through Ethereum.
- Polygon has held its TVL fairly consistently, and this can be interpreted as meaning that users tend to stay within the Polygon ecosystem, or that there is an equal amount being bridged in and out. Other EVM chains such as Avalanche and Fantom have experienced a significant drop off, which would mean a decrease of capital 'rotating' into these chains, as users may not see value in participating in those ecosystems.

In [6]:
# Connect to the DuneAnalytics client
DUNE_USER = 'tarcer'
DUNE_PASSWORD = '1millionLFG!'

# initialize client
dune = DuneAnalytics(DUNE_USER, DUNE_PASSWORD)

# login
dune.login()

# fetch token
dune.fetch_auth_token()

#get results by query id
result_id = dune.query_result_id(query_id=991592)

# fetch query result
data = dune.query_result(result_id)

#data

In [7]:
data_results = data["data"]["get_result_by_result_id"]

data_list = []

# loop through result dictionaries, append to a list and create a dataframe
for stats in data_results:
    daily_bridge_data = stats["data"]
    data_list.append(daily_bridge_data)

df_dune = pd.DataFrame(data_list)
df_dune


,bridge,day,tvl_usd
0,Harmony Bridges,2022-06-06T00:00:00+00:00,1.153384e+08
1,Harmony Bridges,2021-09-22T00:00:00+00:00,6.327543e+07
2,Harmony Bridges,2021-10-13T00:00:00+00:00,1.387668e+08
3,Harmony Bridges,2022-04-25T00:00:00+00:00,3.178318e+08
4,Harmony Bridges,2022-04-02T00:00:00+00:00,4.743433e+08
...,...,...,...
2793,Avalanche Bridge,2021-09-19T00:00:00+00:00,1.911859e+09
2794,Avalanche Bridge,2022-06-27T00:00:00+00:00,1.326026e+09
2795,Avalanche Bridge,2022-01-03T00:00:00+00:00,4.006812e+09
2796,Avalanche Bridge,2022-03-21T00:00:00+00:00,4.644645e+09


In [8]:
# plot 

fig = px.area(df_dune, x="day",
             y="tvl_usd",
             title="Cross-chain Bridge TVL",
             color="bridge")

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b> Bridge TVL </b> (USD)")


fig.show()

# Metric 3. Metapool staking via Dapplooker

Metapool is the inhouse dApp for liquid staking in Near. In return for staking NEAR tokens, Metapool provides stNEAR which can be utilized across a number of dApps in the Near ecosystem. Liquid staking is a very popular approach to managing assets, as a user is able to recieve attractive yields from staking while still maintaining access to their capital, allowing them to participate in the market via stNEAR.

The datasource utilized is DappLooker, which is one of the only platforms that supports data for the Near chain. In this scenerio I have built a dataset in DappLooker, and use an HTTP get request to extract the response data.

Chart: 
- Showcases the daily amount of new NEAR stakes compared to the amount that was unstaked. The total staked NEAR is displayed in the background as an area figure.
- A significant amount of NEAR is being unstaked every day, with a very miniscule amount of new staking taking its place. This downtrend can also be seen in the gradually decline in the total NEAR staked. 
 
 Dapplooker link: https://analytics.dapplooker.com/question/1202-network-metrics-tc

In [16]:
def get_dapplooker():

    HEADERS = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
        }

    dl_url = "http://analytics.dapplooker.com/public/question/1507c4db-53a2-4c79-8077-e7eb470d6b83.json"
    

    try:
        response = requests.get(dl_url, headers=HEADERS, timeout=60).json()
    except request.ConnectionError as e:
        logging.error(f"There was an error with the request, {e}")
        sys.exit(1)
        

    df = pd.DataFrame(response)

    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df.set_index('Timestamp', inplace=True)
    df.index = df.index.date
    df.sort_index(ascending=False, inplace=True)
    
    return df

full_dl_data = get_dapplooker()
full_dl_data.head(10)

,Ref Meta St Near Apr,Ref Meta Price,Near Usd Price,St Near Price Usd,Aurora St Near Price,Aurora Refill Balance Eth,Max Meta Rewards Stakers,St Near Price,Ref Meta Price Usd,Aurora Withdraw Balance Eth,...,Max Meta Rewards Lp,Nslp Share Price,Operator Balance Near,Staking Pools Count,Nslp Total Shares,Total Stake Shares,Trisolaris X T R I St N E A R Supply,Env Epoch Height,Min Deposit Amount,Trisolaris St N E A R W Near Supply
2022-07-06,19.75,0.001330,3.397357,3.730329,1.098009,0.099865,1000000.0,1.098009,0.004960,0.199767,...,3500000.0,1.2285,33.6560,104.0,86695.3039,7.933981e+06,26447850.33,1374.0,1.0,7.849303e+10
2022-07-05,18.59,0.001317,3.534508,3.880211,1.097808,0.099865,1000000.0,1.097808,0.005111,0.199767,...,3500000.0,1.2284,33.9265,104.0,88024.3288,7.933972e+06,26494061.51,1373.0,1.0,7.882955e+10
2022-07-04,16.07,0.001331,3.317374,3.640491,1.097402,0.099865,1000000.0,1.097402,0.004844,0.199767,...,3500000.0,1.2284,37.1601,104.0,87177.9746,7.937521e+06,26676735.35,1371.0,1.0,8.603192e+10
2022-07-03,15.59,0.001345,3.319411,3.641676,1.097085,0.099865,1000000.0,1.097085,0.004899,0.199767,...,3500000.0,1.2283,37.7048,101.0,84560.9230,7.937425e+06,26676735.35,1370.0,1.0,8.602088e+10
2022-07-02,15.62,0.001338,3.351027,3.675453,1.096814,0.099865,1000000.0,1.096814,0.004917,0.199767,...,3500000.0,1.2282,37.9924,101.0,84004.0674,7.939631e+06,26676735.62,1368.0,1.0,8.601219e+10
2022-07-01,30.45,0.001340,3.330739,3.651855,1.096410,0.099865,1000000.0,1.096410,0.004895,0.199767,...,3500000.0,1.2281,38.5968,101.0,85266.7566,7.942332e+06,27486078.21,1366.0,1.0,8.640343e+10
2022-06-30,29.75,0.001358,3.408986,3.737022,1.096227,0.099865,1000000.0,1.096227,0.005074,0.199767,...,3500000.0,1.2281,38.8670,101.0,88093.8572,7.967867e+06,27486078.62,1365.0,1.0,8.679380e+10
2022-06-29,30.06,0.001355,3.414873,3.742087,1.095820,0.099865,1000000.0,1.095820,0.005070,0.199767,...,3500000.0,1.2280,39.4468,101.0,89112.7525,8.273768e+06,27486078.31,1363.0,1.0,8.723898e+10
2022-06-28,30.15,0.001349,3.730287,4.086177,1.095405,0.099865,1000000.0,1.095405,0.005514,0.199767,...,3500000.0,1.2280,40.8574,101.0,97661.8314,8.275021e+06,27522400.83,1362.0,1.0,8.741305e+10
2022-06-27,29.38,0.001369,3.789529,4.150321,1.095208,0.099865,1000000.0,1.095208,0.005682,0.199767,...,3500000.0,1.2280,42.9447,101.0,98519.9494,8.296710e+06,27522400.83,1360.0,1.0,8.735175e+10


In [17]:
# plot

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces

fig.add_trace(
    go.Scatter(x=full_dl_data.index, y=full_dl_data["Total Actually Staked"], 
               name="Total NEAR staked", 
               stackgroup = 'one',
               mode = 'lines',
               line=dict(width=0.5, color='lightblue')),
    secondary_y=True,
)


fig.add_trace(
    go.Bar(x=full_dl_data.index, y=full_dl_data["Total Unstake Claims"], name="Amount NEAR unstaked"),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(x=full_dl_data.index, y=full_dl_data["Epoch Stake Orders"], name="New NEAR staked"),
    secondary_y=False,
)


# Add figure title
fig.update_layout(
    title_text="NEAR Liquid staking in Metapool"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b> New Stake/Unstake Orders </b> (NEAR)", secondary_y=False)
fig.update_yaxes(title_text="<b> Currently staked </b> (NEAR)", secondary_y=True)

fig.show()

# Exploring subgraphs

NEAR is a Non-EVM chain, so most major data providers do not cover it. Specific metrics are more difficult to access, but there are other tools that can be utilized to retrieve harder-to-retrieve datasets. 

The Graph is currently developing subgraphs to index the Near chain and allow for more widely-available data retrival. Currently, they are still in the process of developing these subgraphs, but there are community-created subgraphs that can be queried.

Ths subgraph being explored here only contains data on the creation of new accounts. This is an exercise in showcasing how data can be retrieved via a graphQL query and python.

subgraph used: https://thegraph.com/hosted-service/subgraph/chainscore/near-subgraph?query=Example%20query

In [19]:
def run_query(s_query, http_query='https://api.thegraph.com/subgraphs/name/chainscore/near-subgraph'):

    # endpoint where you are making the request
    request = requests.post(http_query, json={'query': s_query})
    if request.status_code == 200:
        return request.json()

# grahpQL query
accounts = """
{
  accounts(first: 1000, skip: %d, orderBy: createdOn, orderDirection: desc ){
    id
    publicKey
    claimed
    createdOn
    }
  }
"""

df_accounts = pd.DataFrame()

for i in range(7):
    
    # The `skip` argument must be between 0 and 5000
    skip = ((i-1) * 1000) if i > 0 else 0

    result = run_query((accounts % skip))
    
    # From the nested json, extract the 'account' dictionary
    account_data = result["data"]["accounts"]
    
    account_created_df = pd.DataFrame(account_data)
    
    #print(account_created_df.head(5))
    
    df_accounts = df_accounts.append(account_created_df)
    
    
df_accounts

/var/folders/6t/s6dddj4s3cq79crcd2s_d_h40000gn/T/ipykernel_70126/3756855442.py:36: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/6t/s6dddj4s3cq79crcd2s_d_h40000gn/T/ipykernel_70126/3756855442.py:36: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/6t/s6dddj4s3cq79crcd2s_d_h40000gn/T/ipykernel_70126/3756855442.py:36: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/6t/s6dddj4s3cq79crcd2s_d_h40000gn/T/ipykernel_70126/3756855442.py:36: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/6t/s6dddj4s3cq79crcd2s_d_h40000gn/T/ipykernel_70126/3756855442.py:36: FutureWarning:

The frame.append method is de

,id,publicKey,claimed,createdOn
0,0988.near,7wsa9VdmNKzF696DW49fg7LY5wBMhvjxBTWRfiSeK87,False,1657171843329527109
1,0855.near,8yr7Ur72jpJq39dN8nMjFFENeckipWkkxFoTXMkDaewu,False,1657171736775637394
2,0844.near,HUrJucvuwye5mGhBjv568paRRVELyWoU9uDeMhzfHFF8,False,1657171656453725514
3,0833.near,CoKY3goKU6v3A84U2oxnuWkr1Q62DXmGRh44eMr9x2vx,False,1657171596098180657
4,0788.near,Da3ug7NPcEPNpDfwrGnS6BdUPw7NUNhS8Z2mJ9o1mgLR,False,1657171489336877805
...,...,...,...,...
995,dguhrpmwqhuqtvoduymwtjhtkhcscblsnaybxrousdemtc...,ed25519:CB3GExJqarDrdUkR8XqjK2ggNHMhBYeFq1b2Cp...,False,1655940894749550093
996,edeoazkhlhzpmwcytaqiwiklyfaqxaihckqywfyxvsbgop...,ed25519:5H2uqzYET84q9XEZaQgBEgJANFgfPy5SQ1U9UE...,False,1655940892301162128
997,bhzefoswswcpsbruydwbozeauurfckohbojwmajtcgwryn...,ed25519:BG2kU1pWYhvQG4WdXtDURokiBGjuTkfvPqBS1i...,False,1655940886856012532
998,vhfjndplfyliuvpchjivecammhoelnnblypbtpdvqihsxw...,ed25519:HVQ9GsPNcDksMh4dQro7z4xvgNRtY8fxuJtEr9...,False,1655940860265455640


In [20]:
df_accounts['createdOn'] = df_accounts['createdOn'].astype(int)

df_accounts['date'] = pd.to_datetime(df_accounts['createdOn'], unit='ns').apply(lambda x: x.to_datetime64())

# df_accounts['date'] = df_accounts['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
# df_accounts_count = df_accounts.groupby(['date'])['date'].count()
# df_accounts_count_1 = pd.DataFrame(df_accounts_count)

df_accounts_count = df_accounts.groupby(df_accounts['date'].dt.date).size().reset_index(name='Count')

df_accounts_count

,date,Count
0,2022-06-22,186
1,2022-06-23,2217
2,2022-06-24,991
3,2022-06-25,284
4,2022-06-26,195
5,2022-06-27,155
6,2022-06-28,242
7,2022-06-29,171
8,2022-06-30,228
9,2022-07-01,134


In [21]:
fig = px.line(df_accounts_count, x="date", y="Count", title='New NEAR accounts')
fig.show()